# Lesson 01: Getting topographic data

*This lesson made by Simon M Mudd and last updated 21/11/2021*

In this lesson we will grab some topographic data from the internet using [Opentopography](https://opentopography.org/).

[Opentopography](https://opentopography.org/) hosts a large number of topographic datasets and is supported by the National Science Foundation of the United States. 

## What you can find on OpenTopography

You can see what is on OpenTopograhy by [following this link](https://portal.opentopography.org/dataCatalog)

But briefly it has:
    
* Several global topographic datasets. 
    * Shuttle Radar Topography Mission (SRTM) data, at approximately 30 and 90 m grid cell size. Based on radar data. 
    * The NASADEM, which is a reprocessed version of the SRTM data (so theoretically better quality than SRTM from the same base data).
    * The ALOS World 3D 30 m dataset. This is based on optical data made using photogrammetry from the ALOS satellites. It is a downscaled version of the 5 m dataset (the difference is that the 30 m one is free).
    * The Copernicus DEM, which is 30m grid comes from a variety of sources but much of it from the TanDEM-X radar satellites.
* Vast quantities of high resolution data from lidar (cm scale point clouds and 1 m grid cell size DEMs).
    * These data are mostly from the United States but the site also has selected data from around the world (for example there are a number of datasets from New Zealand)

OpenTopography has an api for its global dataset so it is, by some wide margin, the most convenient place to source global data at 30 m grid spacing or coarser. 

Here is the link to the [api key](https://portal.opentopography.org/apidocs/#/Public/getGlobalDem)

However, we will use a python package called `lsdviztools` to grab this data. `lsdviztools` is a python package written at the University of Edinburgh that has various scripts and tools for manipulating and visualising topographic data and derivative datasets. 

## First import some stuff we need

We need to update a package called `lsdviztools` that has a subcomponent for grabbing opentopography data. 

In [ ]:
!pip install lsdviztools --upgrade

Double check that the version is correct. For this lesson we need version 0.4.7 or above.

In [ ]:
import lsdviztools
lsdviztools.__version__

Now import a bunch of other stuff for the lesson

In [ ]:
import lsdviztools.lsdbasemaptools as bmt
import rasterio as rio
import matplotlib.pyplot as plt
import numpy as np

## Grab some data

Okay, lets grab some data! We use a tool in `lsdviztools.lsdbasemaptools` called the `ot_scraper` (`ot` is for opentopography.org).

Below is some code for grabbing data from opentopography. It uses the syntax:

`DEM_name = bmt.ot_scraper(parameters=...)`

where you need to give it some paramaters. These are things like what sort of data you want (most people will use the SRTM 30 metre data) and you also tell it the lower left and the upper right corners, using latitude and longitude. You can get these from Google Maps by right clicking on the map and selecting "what's here". 

In this example I use the Rio Aguas in southern Spain (a place we run field courses). 

We designate the parameters by giving the lower left and upper right coordinates of the area from which you want the DEM. 

The format of these is so that you can right click on google maps and click on the coordinates, which will copy them to your clipboard, and you can paste them in to the lower left and upper right coordinates.

These coordinates are lists so you need them in square brackets. 

In [ ]:
lower_left = [36.990554387425014, -2.318307057720176]
upper_right = [37.23367133834253, -1.8425313329873874]

Okay, below is the syntax for downloading the data.

In [ ]:
Aguas_DEM = bmt.ot_scraper(source = "SRTMGL1",
                           lower_left_coordinates = lower_left, 
                           upper_right_coordinates = upper_right,
                           prefix = "rio_aguas") 
Aguas_DEM.print_parameters()
Aguas_DEM.download_pythonic()

The options for the data types are:
    
* `SRTMGL3` = (SRTM GL3 90m)
* `SRTMGL1` = (SRTM GL1 30m)
* `SRTMGL1_E` = (SRTM GL1 Ellipsoidal 30m)
* `AW3D30` = (ALOS World 3D 30m)
* `AW3D30_E` = (ALOS World 3D Ellipsoidal, 30m)
* `SRTM15Plus` = (Global Bathymetry SRTM15+ V2.1)
* `NASADEM` = (NASADEM Global DEM) *
* `COP30` = (Copernicus Global DSM 30m) *
* `COP90` = (Copernicus Global DSM 90m) *

You will need to create an [account at opentopography](https://portal.opentopography.org/login) to generate an API key for the datasets with an `*` (that is, `NASADEM`, `COP30` and `COP90`). To enter the api key you add a parameter to the `ot_scraper`: `api_key = ` and then your api key. 

## Looking at the data

Okay, lets look at the data. We can always see what is in the current directory by calling the linux command `ls`, which is short for "list". It tells you what is in your directory. But in a python notebook, any commands to the underlying linux system need to be preceded with a `!` symbol (we sometimes call this "bang"). So lets see what we have:

In [ ]:
!ls

Hey, look, there is the file `rio_aguas_SRTMGL1.tif`! This file happens to be a GeoTIFF, which is a kind of raster data format. A raster is just a bunch of numbers in a grid, which you can read about here: [Raster images](https://en.wikipedia.org/wiki/Raster_graphics)
Geospatial rasters are just raster images with some georeferencing attached. 

### Getting raster information (crs and dimensions) using GDAL

Okay, we want to know some information about this raster. 

If you spend a lot of time with this kind of data you will be working in linux terminals, and you will almost certainly use a package called *gdal* (the geospatial data abstraction library) to look at files. [gdal](https://gdal.org/index.html) has python bindings, but the simplest way to use it is through its command line interface. That is, we use some linux command to access *gdal*. 

The most basic one is *gdalinfo* which just gives you information about your raster. This sits in linux so you need 1) gdal installed and 2) to call the linux system with the `!` symbol. 

Lets do that:

In [ ]:
!gdalinfo rio_aguas_SRTMGL1.tif

Whoa! That is a lot of information!! What is in here??

* The raster format (GeoTIFF) is on this line:
 `Driver: GTiff/GeoTIFF`

* There is the raster size on this line:
 `Size is 1713, 875`

* After that there are a bunch of lines explaining the **CRS**, or coordinate reference system. **THE CRS IS VERY IMPORTANT**. It tells any geospatial tool (e.g. a GIS, or some geospatial corner of python) where your data is. **Without the CRS your data is just an array of numbers without any spatial context.*

* You don't need to worry too much about the details of the **CRS** other than to know what it is. It so happens that these have a shorthand code, called the **EPSG** code. You can see it here in amongst all the other georeferencing information: `ID["EPSG",4326]`

* It tells you the upper left corner location: `Origin = (-2.318472222198693,37.233749999996988)`

* The size of the pixels: `Pixel Size = (0.000277777777778,-0.000277777777778)` More on this later.

* Then, there are lines about the spatial extent that follow: `Corner Coordinates`

* It tells you what the data type is: `Type=Int16` (in this case all the numbers in the DEM are integers, which is important)

* And it tells you the value of nodata pixels: `NoData Value=-32768`

### Getting raster information using rasterio

We can also get raster information using a python package called `rasterio`. It is used for processing rasters. One problem with this approach is you need to actually load the raster (gdalinfo just reads the metadata), so it is a bit slower than gdalinfo. But for small rasters (say, less than 20Mb) you probably won't notice any difference. 

Here is how you load a raster using `rasterio`

In [ ]:
filename = "rio_aguas_SRTMGL1.tif"
with rio.open(filename) as src:
    print(src.meta)
    

You can see that this has some of the same information as `gdalinfo` but in a terse format. We can get specific information from the raster with the following lines:

In [ ]:
filename = "rio_aguas_SRTMGL1.tif"
with rio.open(filename) as src:
    print("Raster crs is:")
    print(src.crs)
    print("\nRaster bounding box is:")
    print(src.bounds)
    print("\nRaster pixel sizes in the X,Y directions are:")
    print(src.res)

## Looking at your raster

We can look at your raster using the `imshow` function in `matplotlib`:

In [ ]:
%matplotlib inline
filename = "rio_aguas_SRTMGL1.tif"
with rio.open(filename) as src:
    plt.imshow(src.read(1), cmap='pink')
    plt.show()

We can also generate a hillshade (using the ever-useful gdal) and look at that. You need to give it the input and output filenames, and I specify an algorithm that is better than the default. 

In [ ]:
!gdaldem hillshade rio_aguas_SRTMGL1.tif rio_aguas_SRTMGL1_HS.tif -alg ZevenbergenThorne

In [ ]:
%matplotlib inline
filename = "rio_aguas_SRTMGL1_HS.tif"
with rio.open(filename) as src:
    plt.imshow(src.read(1), cmap='pink')
    plt.show()

This hillshade looks a bit funny and in the next lesson I will explain why. 

## What you should have learned and potential modifications

After this lesson you should:
    
* Know what opentopography is.
* Know how to download a raster from opentopography.
    * Try to download a raster in a different place.
    * Try to download a raster from a different data source.
* Check the raster metadata using gdal or rasterio.
* Make a simple plot of the raster, including a hillshade made with gdal